In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=a95a37567693230db2dd2d0b7a82d25de5cff7e6cbfbff97efc3232bbca535cb
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [ ]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/strength-programmer/Day-4-AI-First-Dataset-Julian/refs/heads/main/parcel_information.csv')


In [5]:
df.sample(5)

,Parcel ID,Sender Name,Receiver Name,Sender Address,Receiver Address,Parcel Weight (kg),Parcel Dimensions (cm),Shipping Date,Expected Delivery Date,Shipping Method,Current Location,Status,Tracking Number,Delivery Confirmation,Delivery Time,Shipping Cost (USD)
6,P123456,Prime Delivery,Linda Lee,"60 Harbor Dr, City D","78 Town Dr, City G",1.4,15x10x8,11/16/2024,11/23/2024,Standard,City G,Delivered,TRK123456,Yes,11:00,18
9,P123459,SpeedyExpress,Chris Pine,"45 Elm St, City J","2000 Mountain Rd, City D",2.0,25x20x15,11/19/2024,11/28/2024,Standard,City D,Delivered,TRK123459,Yes,10:30,15
2,P123452,Alice Lee,Bob Brown,"456 Another St, City E","789 Another Rd, City F",3.0,25x20x12,11/12/2024,11/18/2024,Express,City F,Delayed,TRK123452,No,NaN,20
0,P123450,ABC Corp,John Doe,"123 Sender St, City A","456 Receiver Rd, City B",5.2,30x20x15,11/10/2024,11/15/2024,Express,City B,Delivered,TRK123450,Yes,14:30,25
5,P123455,FastTrack,Tom Hanks,"910 West Ln, City F","10 Plaza Ave, City H",6.2,50x40x35,11/15/2024,11/21/2024,Express,City H,In Transit,TRK123455,No,NaN,22


## Understanding the Dataset

In [6]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info, "\n")



Parcel ID                  object
Sender Name                object
Receiver Name              object
Sender Address             object
Receiver Address           object
Parcel Weight (kg)        float64
Parcel Dimensions (cm)     object
Shipping Date              object
Expected Delivery Date     object
Shipping Method            object
Current Location           object
Status                     object
Tracking Number            object
Delivery Confirmation      object
Delivery Time              object
Shipping Cost (USD)         int64
dtype: object 



In [7]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)



Categorical Features: Index(['Parcel ID', 'Sender Name', 'Receiver Name', 'Sender Address',
       'Receiver Address', 'Parcel Dimensions (cm)', 'Shipping Date',
       'Expected Delivery Date', 'Shipping Method', 'Current Location',
       'Status', 'Tracking Number', 'Delivery Confirmation', 'Delivery Time'],
      dtype='object')
Numerical Features: Index(['Parcel Weight (kg)', 'Shipping Cost (USD)'], dtype='object')


In [9]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
  uniques = df[col].unique().tolist() # get unique values and turn them into lists
  if len(uniques) > max_display: # if there are more than 10 unique values
    return uniques[:max_display] + ["..."]
  return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
  print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Parcel ID: ['P123450', 'P123451', 'P123452', 'P123453', 'P123454', 'P123455', 'P123456', 'P123457', 'P123458', 'P123459', '...']
 - Sender Name: ['ABC Corp', 'XYZ Inc.', 'Alice Lee', "John's Mart", 'QuickShip', 'FastTrack', 'Prime Delivery', 'BlueSky Shipping', 'TopTrans Logistics', 'SpeedyExpress', '...']
 - Receiver Name: ['John Doe', 'Jane Smith', 'Bob Brown', 'Michael Clark', 'Sara Wong', 'Tom Hanks', 'Linda Lee', 'Henry Gold', 'Anna Kim', 'Chris Pine', '...']
 - Sender Address: ['123 Sender St, City A', '789 Sender Blvd, City C', '456 Another St, City E', '100 Ship Rd, City H', '55 Fast Ave, City G', '910 West Ln, City F', '60 Harbor Dr, City D', '222 Cargo Rd, City B', '90 Lake St, City I', '45 Elm St, City J', '...']
 - Receiver Address: ['456 Receiver Rd, City B', '101 Receiver Ave, City D', '789 Another Rd, City F', '67 Cloud St, City A', '99 Star Rd, City C', '10 Plaza Ave, City H', '78 Town Dr, City G', '400 Garden Rd, City E', '150

In [11]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")



Descriptive Statistics for Numerical Features:
        Parcel Weight (kg)  Shipping Cost (USD)
count           20.000000            20.000000
mean             4.630000            23.950000
std              2.292115             9.444993
min              1.400000            10.000000
25%              2.875000            17.250000
50%              4.450000            23.500000
75%              6.050000            30.000000
max              9.200000            45.000000 



## Generating the Template

In [12]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"


    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   - {stat_name}: {value}\n"


    # Define the system prompt
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""


    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


Output only the template without any explanation or introduction.
The template's variables will be dynamically replaced so make sure they're formatted properly."""


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None


In [13]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"Parcel ID {Parcel ID}, sent by {Sender Name} from {Sender Address} to {Receiver Name} at {Receiver Address}, weighs {Parcel Weight (kg)} kg and has dimensions of {Parcel Dimensions (cm)}. It was shipped on {Shipping Date} using the {Shipping Method} method, and is currently located in {Current Location}. The parcel's status is {Status}, with a tracking number of {Tracking Number}. Delivery confirmation is {Delivery Confirmation}, and it is expected to arrive by {Expected Delivery Date}. The delivery occurred at {Delivery Time} and the shipping cost was ${Shipping Cost (USD)}."


## Populating the Template with Actual Values

In [14]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    paragraph = template.format(**row_dict)

    # Generate the populated template per row
    return paragraph


In [15]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

In [16]:
df['paragraph'][0]

'"Parcel ID P123450, sent by ABC Corp from 123 Sender St, City A to John Doe at 456 Receiver Rd, City B, weighs 5.2 kg and has dimensions of 30x20x15. It was shipped on 11/10/2024 using the Express method, and is currently located in City B. The parcel\'s status is Delivered, with a tracking number of TRK123450. Delivery confirmation is Yes, and it is expected to arrive by 11/15/2024. The delivery occurred at 14:30 and the shipping cost was $25."'

In [17]:
df.sample(5)

,Parcel ID,Sender Name,Receiver Name,Sender Address,Receiver Address,Parcel Weight (kg),Parcel Dimensions (cm),Shipping Date,Expected Delivery Date,Shipping Method,Current Location,Status,Tracking Number,Delivery Confirmation,Delivery Time,Shipping Cost (USD),paragraph
6,P123456,Prime Delivery,Linda Lee,"60 Harbor Dr, City D","78 Town Dr, City G",1.4,15x10x8,11/16/2024,11/23/2024,Standard,City G,Delivered,TRK123456,Yes,11:00,18,"""Parcel ID P123456, sent by Prime Delivery fro..."
17,P1234517,EasySend,Mia Silver,"85 Sunset Blvd, City I","101 Rose Ln, City B",1.8,15x10x10,11/27/2024,12/10/2024,Standard,City B,Delayed,TRK1234517,No,NaN,14,"""Parcel ID P1234517, sent by EasySend from 85 ..."
10,P1234510,UrbanCourier,Natalie King,"300 Ocean Rd, City A","500 Beach Ave, City F",5.5,35x25x20,11/20/2024,11/30/2024,Standard,City F,In Transit,TRK1234510,No,NaN,35,"""Parcel ID P1234510, sent by UrbanCourier from..."
11,P1234511,MetroParcel,Liam Fox,"150 Maple Ave, City E","75 Park Blvd, City G",2.8,20x15x12,11/21/2024,12/1/2024,Express,City G,Delivered,TRK1234511,Yes,12:15,12,"""Parcel ID P1234511, sent by MetroParcel from ..."
18,P1234518,ShipNow,Ethan Gray,"50 Meadow St, City H","700 Stone Rd, City E",9.2,65x50x45,11/28/2024,12/12/2024,Express,City E,In Transit,TRK1234518,No,NaN,45,"""Parcel ID P1234518, sent by ShipNow from 50 M..."


## Saving the Populated Templates (Natural Language Texts)

In [18]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [19]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [20]:
# Save the paragraphs to both text and PDF files

save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='NLG.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='NLG.pdf')

All content has been saved as 'NLG.txt'.

All content has been saved as 'NLG.pdf'.

